# Merging datasets

# Import required packages

In [4]:
import pandas as pd
from functools import reduce
from IPython.display import IFrame 

# Read input files from github

Read data of every team member from github.

In [6]:
location ='https://raw.githubusercontent.com/Public-Policy-COVID/'
fileEmma = location+'Emma-li/main/emma_data.csv'
fileBryn = location + 'Bryn-Bandt-Law/main/bryn.bl.clean.data.csv'
fileAyushi = location + 'Ayushi-Dhar/main/ayushi_dhar.csv'
fileNick = location + 'Nick-Maue/main/westcoastrace_clean_data.csv'
emma_data = pd.read_csv(fileEmma)
bryn_data = pd.read_csv(fileBryn)
ayushi_data = pd.read_csv(fileAyushi)
nick_data = pd.read_csv(fileNick)

# Merge Data

Some of the counties in all 3 states did not have hospitals which is why that data set does not have 133 rows (total number of counties in all 3 states) in ayushi_data. In order to ensure that information is reflected, we did a right join on emma_data which has information on all counties.

Merge hospital data with information on covid deaths and total number of deaths per county in Washington, Oregon and California.

In [7]:
merge1 = pd.merge(left = ayushi_data, right = emma_data, left_on = ['Location'], right_on = ['Location'],how = 'right')

# Replace the null values with 0's to reflect 0 number of hospitals and hospital beds in some counties
merge1[['Number_of_beds','Number_of_hospitals']] = merge1[['Number_of_beds','Number_of_hospitals']].fillna(0)

Merge existing information with data on mask useage per county in Washington, Oregon and California.

In [8]:
merge2 = pd.merge(left = merge1, right = bryn_data, left_on = ['Location'], right_on = ['Location'],how = 'inner')

Merge existing information with data on race and population per county in Washington, Oregon and California.

In [9]:
merged_data = pd.merge(left = merge2, right = nick_data, left_on = ['Location'], right_on = ['Location'],how = 'inner')

In [10]:
merged_data

,Number_of_beds,Number_of_hospitals,Location,Urban_Rural_Code,Deaths_COVID,Deaths_total,never,rarely,sometimes,frequently,always,mask_score,total_population,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
0,3667.0,22.0,Alameda_CA,Large central metro,573,10908,0.019,0.008,0.055,0.123,0.795,3.667,1671329.0,49.28,1.06,32.33,11.03,0.94,5.35
1,0.0,0.0,Alpine_CA,NaN,0,0,0.025,0.085,0.088,0.190,0.612,3.279,1129.0,67.94,25.69,1.59,0.35,0.00,4.43
2,52.0,1.0,Amador_CA,Noncore,31,415,0.045,0.013,0.099,0.188,0.655,3.395,39752.0,89.65,2.33,1.67,2.68,0.29,3.38
3,553.0,6.0,Butte_CA,Small metro,101,2313,0.015,0.043,0.111,0.204,0.626,3.381,219186.0,85.65,2.53,5.01,1.90,0.29,4.62
4,25.0,1.0,Calaveras_CA,Noncore,12,385,0.045,0.019,0.098,0.276,0.562,3.291,45905.0,90.95,1.97,1.87,1.08,0.27,3.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,0.0,Wahkiakum_WA,NaN,0,0,0.045,0.057,0.079,0.161,0.658,3.330,4191.0,93.18,0.43,1.43,0.93,0.19,3.84
129,87.0,1.0,Walla Walla_WA,Small metro,38,651,0.083,0.022,0.061,0.193,0.641,3.287,62201.0,91.44,2.18,1.37,1.70,0.40,2.91
130,253.0,1.0,Whatcom_WA,Small metro,63,1881,0.042,0.028,0.061,0.122,0.747,3.504,225300.0,86.46,1.23,3.23,4.48,0.33,4.26
131,50.0,2.0,Whitman_WA,Micropolitan,21,269,0.002,0.009,0.049,0.310,0.629,3.553,50131.0,81.90,1.99,0.84,10.51,0.23,4.52


# Check merged data for errors

In order to ensure that the merge was correct, we are comparing the statistics before and after merging.

In [11]:
merged_data.describe()

,Number_of_beds,Number_of_hospitals,Deaths_COVID,Deaths_total,never,rarely,sometimes,frequently,always,mask_score,total_population,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.00000,133.000000,133.000000,133.000000,133.000000,1.330000e+02,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,885.353383,5.000000,206.000000,2896.030075,0.035128,0.03806,0.071669,0.173594,0.681421,3.427865,3.855370e+05,85.496015,2.592782,4.914286,2.757895,0.383835,3.855714
std,2655.174814,11.189145,761.950905,7680.904490,0.028704,0.03430,0.036789,0.054701,0.095906,0.224187,1.025988e+06,9.305323,3.051460,6.847121,3.086172,0.303019,1.093928
min,0.000000,0.000000,0.000000,0.000000,0.001000,0.00000,0.004000,0.058000,0.305000,2.470000,1.129000e+03,49.280000,0.000000,0.500000,0.230000,0.000000,1.200000
25%,25.000000,1.000000,0.000000,0.000000,0.016000,0.01400,0.048000,0.141000,0.616000,3.301000,2.465800e+04,82.160000,1.180000,1.280000,0.940000,0.210000,3.160000
50%,131.000000,2.000000,22.000000,637.000000,0.026000,0.02800,0.069000,0.168000,0.681000,3.464000,7.948100e+04,88.640000,1.970000,1.920000,1.510000,0.280000,3.720000
75%,553.000000,4.000000,128.000000,2537.000000,0.045000,0.05600,0.091000,0.204000,0.754000,3.591000,2.831110e+05,91.840000,2.860000,5.500000,2.840000,0.450000,4.440000
max,26672.000000,112.000000,8034.000000,75463.000000,0.140000,0.20600,0.213000,0.332000,0.889000,3.822000,1.003911e+07,96.130000,25.690000,39.020000,18.480000,1.710000,7.800000


Since some of the counties did not have hospitals, we are checking to make sure the count after merging has been updated to 133
the new min is 0 and the max values are the same. All the other statistics should be different as we have added extra rows to
reflect 0 number of hospitals and hospital beds.

In [12]:
ayushi_data.describe()

,Number_of_beds,Number_of_hospitals
count,124.000000,124.000000
mean,949.612903,5.362903
std,2739.395143,11.506332
min,6.000000,1.000000
25%,46.750000,1.000000
50%,147.500000,2.000000
75%,723.750000,4.000000
max,26672.000000,112.000000


Since the statistics will be different before and after the merge, we are doing one additional sanity check to ensure all the columns from ayushi_data got added to the merged dataset.

Checking to make sure the statistics after doing an inner join of ayushi_data and merged_data are the same. This ensures all the columns from ayushi_data are present in merged_data.


In [13]:
merge_check = pd.merge(left = ayushi_data, right = merged_data, left_on = ['Location','Number_of_beds','Number_of_hospitals'], right_on = ['Location','Number_of_beds','Number_of_hospitals'],how = 'inner')
merge_check.describe()

,Number_of_beds,Number_of_hospitals,Deaths_COVID,Deaths_total,never,rarely,sometimes,frequently,always,mask_score,total_population,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
count,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,1.240000e+02,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,949.612903,5.362903,220.951613,3106.225806,0.034589,0.037661,0.071629,0.171242,0.684774,3.433742,4.125470e+05,85.155806,2.467500,5.179919,2.910806,0.397258,3.889274
std,2739.395143,11.506332,787.222594,7915.483062,0.027713,0.034799,0.036879,0.052319,0.097540,0.227242,1.057724e+06,9.308718,2.349881,7.018107,3.140917,0.307326,1.109856
min,6.000000,1.000000,0.000000,0.000000,0.001000,0.000000,0.004000,0.058000,0.305000,2.470000,2.221000e+03,49.280000,0.000000,0.500000,0.280000,0.010000,1.200000
25%,46.750000,1.000000,0.000000,0.000000,0.016000,0.013000,0.048000,0.137750,0.623000,3.306500,2.824775e+04,82.027500,1.217500,1.367500,1.062500,0.217500,3.200000
50%,147.500000,2.000000,25.000000,715.000000,0.026000,0.027500,0.069000,0.167000,0.687500,3.466000,9.027000e+04,88.320000,1.970000,2.090000,1.660000,0.285000,3.725000
75%,723.750000,4.000000,153.750000,2847.000000,0.044250,0.056000,0.091250,0.201750,0.754500,3.600250,3.563802e+05,91.047500,2.862500,5.760000,3.337500,0.460000,4.497500
max,26672.000000,112.000000,8034.000000,75463.000000,0.132000,0.206000,0.213000,0.320000,0.889000,3.822000,1.003911e+07,96.130000,18.350000,39.020000,18.480000,1.710000,7.800000


All the statistics for these columns should remain the same.

In [14]:
emma_data.describe()

,Deaths_COVID,Deaths_total
count,133.000000,133.000000
mean,206.000000,2896.030075
std,761.950905,7680.904490
min,0.000000,0.000000
25%,0.000000,0.000000
50%,22.000000,637.000000
75%,128.000000,2537.000000
max,8034.000000,75463.000000


All the statistics for these columns should remain the same.

In [15]:
bryn_data.describe()

,never,rarely,sometimes,frequently,always,mask_score
count,133.000000,133.00000,133.000000,133.000000,133.000000,133.000000
mean,0.035128,0.03806,0.071669,0.173594,0.681421,3.427865
std,0.028704,0.03430,0.036789,0.054701,0.095906,0.224187
min,0.001000,0.00000,0.004000,0.058000,0.305000,2.470000
25%,0.016000,0.01400,0.048000,0.141000,0.616000,3.301000
50%,0.026000,0.02800,0.069000,0.168000,0.681000,3.464000
75%,0.045000,0.05600,0.091000,0.204000,0.754000,3.591000
max,0.140000,0.20600,0.213000,0.332000,0.889000,3.822000


All the statistics for these columns should remain the same.

In [17]:
nick_data.describe()

,total_population,white_total_pct,black_total_pct,aian_total_pct,asian_total_pct,nhopi_total_pct,multiracial_total_pct
count,1.330000e+02,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,3.855370e+05,85.496015,2.592782,4.914286,2.757895,0.383835,3.855714
std,1.025988e+06,9.305323,3.051460,6.847121,3.086172,0.303019,1.093928
min,1.129000e+03,49.280000,0.000000,0.500000,0.230000,0.000000,1.200000
25%,2.465800e+04,82.160000,1.180000,1.280000,0.940000,0.210000,3.160000
50%,7.948100e+04,88.640000,1.970000,1.920000,1.510000,0.280000,3.720000
75%,2.831110e+05,91.840000,2.860000,5.500000,2.840000,0.450000,4.440000
max,1.003911e+07,96.130000,25.690000,39.020000,18.480000,1.710000,7.800000


# Output merged data as csv file

Output the merged data as a csv file and remove the index from the data frame.

In [13]:
merged_data.to_csv('Merged_data.csv',header = True, index = False)